<h1> Introduction/Business Problem

The investigation that will be undertaken will identify the top businesses in the neighbourhoods of Paris. As an entrepeneur this will be vital too see which areas are similar and which businesses would be more popular investing in. A comparison can, therefore, be made as to which neighbourhoods prefer what kind of business in the city of Paris and what the culture is like within the city for an up and coming entrepreneur in the area.

<h1> How the data will be utilised

Using the foursquare API, the city of Paris will be analysed. Initially, the neighbourhoods within Paris will be identified. Subsequently, the top 5 venues within each neighbourhood will be determined. As an entrepreneur the areas can be discussed as to which businesses are more likely to be invested in.

<h1> Methodology

A dataframe will be obtained indicating the top 5 venues and their respective neighbourhoods. The data will be obtained using the Foursquare API through the venues search query. The following section shows the coding methodology

<h1> Coding Methodology and Results

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Finding Paris co-ordinates

In [2]:
address = 'Paris, France'

geolocator = Nominatim(user_agent="Paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 48.8566969, 2.3514616.


In [34]:
CLIENT_ID = 'MHZLZU1IZCVNMNMGKVJT45U2CF2G5IORZT3YC1KNJBCGNS2H' # your Foursquare ID
CLIENT_SECRET = 'OVMZKCVWPJXMAWZ45HWBEKWQF1GRINVGNPARALPSJYVGOAQZ' # your Foursquare Secret
ACCESS_TOKEN = '43VNL3RHVISRZRGCICR3ABKWLGMPKQLJJPKIBJIIVBTDN0XB' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MHZLZU1IZCVNMNMGKVJT45U2CF2G5IORZT3YC1KNJBCGNS2H
CLIENT_SECRET:OVMZKCVWPJXMAWZ45HWBEKWQF1GRINVGNPARALPSJYVGOAQZ


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
search_query = 'Top Venues'
radius = 5000

In [38]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MHZLZU1IZCVNMNMGKVJT45U2CF2G5IORZT3YC1KNJBCGNS2H&client_secret=OVMZKCVWPJXMAWZ45HWBEKWQF1GRINVGNPARALPSJYVGOAQZ&ll=48.8566969,2.3514616&v=20180605&query=Top Venues&radius=5000&limit=100'

Finding venues in Paris and locations

In [39]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6097eaf5bc94a66b17ec9c93'},
 'response': {'venues': [{'id': '4fbf72e6e4b0599987197344',
    'name': 'Copy-Top',
    'location': {'address': '78 boulevard de Sébastopol',
     'crossStreet': 'Rue Greneta',
     'lat': 48.8650712,
     'lng': 2.3521058999999696,
     'labeledLatLngs': [{'label': 'display',
       'lat': 48.8650712,
       'lng': 2.3521058999999696}],
     'distance': 933,
     'postalCode': '75003',
     'cc': 'FR',
     'city': 'Paris',
     'state': 'Île-de-France',
     'country': 'France',
     'formattedAddress': ['78 boulevard de Sébastopol (Rue Greneta)',
      '75003 Paris',
      'France']},
    'categories': [{'id': '4bf58dd8d48988d1f4941735',
      'name': 'Design Studio',
      'pluralName': 'Design Studios',
      'shortName': 'Design',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/design_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '518635464'},
    'referralId': 

In [50]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [79]:
venues = results['response']['venues']

In [76]:
venues.columns

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.address',
       'location.crossStreet', 'location.lat', 'location.lng',
       'location.labeledLatLngs', 'location.distance', 'location.postalCode',
       'location.cc', 'location.city', 'location.state', 'location.country',
       'location.formattedAddress', 'venuePage.id'],
      dtype='object')

In [87]:
venues2 = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name','categories' , 'location.lat', 'location.lng']
venues2 = venues2.loc[:, filtered_columns]

# filter the category for each row
venues2['categories'] = venues2.apply(get_category_type, axis=1)

# clean columns
venues2.columns = [col.split(".")[-1] for col in venues2.columns]

venues2.head()

,name,categories,lat,lng
0,Copy-Top,Design Studio,48.865071,2.352106
1,COPY-TOP Louvre - Etienne Marcel / Imprimerie ...,Design Studio,48.865900,2.343250
2,Top Time,Jewelry Store,48.856384,2.356865
3,Copy-Top,Paper / Office Supplies Store,48.865992,2.343304
4,Top Souvenirs,Gift Shop,48.852809,2.346204


<h1> Explore Neighbourhoods in Paris

In [88]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [90]:
paris_venues = getNearbyVenues(names=venues2['name'],
                                   latitudes=venues2['lat'],
                                   longitudes=venues2['lng']
                                  )

Copy-Top
COPY-TOP Louvre - Etienne Marcel / Imprimerie Paris 2ème
Top Time
Copy-Top
Top Souvenirs
COPY-TOP Pyramides - Palais Royal / Imprimerie Paris 1er
Top Sexy
COPY-TOP République
Top Shelf
COPY-TOP Bonne Nouvelle / Imprimerie Paris 10ème
COPY-TOP Opéra - Bourse
Tours de la Cathédrale Notre-Dame de Paris
COPY-TOP Gare de Lyon - Bastille / Imprimerie Paris 12ème
Elie Top France
COPY-TOP Raspail - Rennes / Imprimerie Paris 6ème
Töp Yög
Copy-Top
Top Retouches
COPY-TOP Papillon - La Fayette / Imprimerie Paris 9ème
Top Fruits
Top Sushi
COPY-TOP Le Peletier - Châteaudun / Imprimerie Paris 9ème
COPY-TOP Madeleine / Imprimerie Paris 9ème
Copy-Top
Roof Top Lounge
COPY-TOP Saint Lazare - Rome / Imprimerie Paris 8ème
COPY-TOP Mogador - Trinité / Imprimerie Paris 9ème
COPY-TOP Montparnasse / Imprimerie Paris 15ème
COPY-TOP Malesherbes - Saint Augustin / Imprimerie Paris 8ème
Au Top
Top Boutique
Top Scooter
Copy-Top
COPY-TOP Voltaire / Imprimerie Paris 11ème
Top Form
Au Top
Jardins du Carrousel

In [92]:
paris_venues.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

<h1> Venues returned for each neighbourhood

In [93]:
paris_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Au Top,119,119,119,119,119,119
COPY-TOP Batignolles - Rome / Imprimerie Paris 17ème,38,38,38,38,38,38
COPY-TOP Bonne Nouvelle / Imprimerie Paris 10ème,100,100,100,100,100,100
COPY-TOP Gare de Lyon - Bastille / Imprimerie Paris 12ème,95,95,95,95,95,95
COPY-TOP Haussmann - Miromesnil / Imprimerie Paris 8ème,38,38,38,38,38,38
COPY-TOP Le Peletier - Châteaudun / Imprimerie Paris 9ème,72,72,72,72,72,72
COPY-TOP Louis Blanc - La Fayette / Imprimerie Paris 10ème,65,65,65,65,65,65
COPY-TOP Louvre - Etienne Marcel / Imprimerie Paris 2ème,100,100,100,100,100,100
COPY-TOP Madeleine / Imprimerie Paris 9ème,100,100,100,100,100,100


In [94]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighborhood'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal Lock,Candy Store,Caucasian Restaurant,Ch'ti Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Resort,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Copy-Top,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [96]:
paris_grouped = paris_onehot.groupby('Neighborhood').mean().reset_index()
paris_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal Lock,Candy Store,Caucasian Restaurant,Ch'ti Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Resort,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Train Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Au Top,0.000000,0.000000,0.008403,0.000000,0.000000,0.000000,0.000000,0.016807,0.000000,0.000000,0.000000,0.008403,0.000000,0.00,0.008403,0.000000,0.033613,0.033613,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.025210,0.008403,0.008403,0.008403,0.000000,0.000000,0.0

<h1> Obtaining Top 5 Venues 

In [97]:
num_top_venues = 5

for hood in paris_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Au Top----
               venue  freq
0  French Restaurant  0.07
1               Café  0.05
2    Thai Restaurant  0.03
3        Coffee Shop  0.03
4       Burger Joint  0.03


----COPY-TOP Batignolles - Rome / Imprimerie Paris 17ème----
                venue  freq
0   French Restaurant  0.11
1            Wine Bar  0.11
2               Hotel  0.08
3  Italian Restaurant  0.05
4              Bistro  0.05


----COPY-TOP Bonne Nouvelle / Imprimerie Paris 10ème----
                 venue  freq
0    French Restaurant  0.11
1                Hotel  0.10
2             Wine Bar  0.05
3  Japanese Restaurant  0.05
4          Pizza Place  0.04


----COPY-TOP Gare de Lyon - Bastille / Imprimerie Paris 12ème----
               venue  freq
0              Hotel  0.14
1  French Restaurant  0.11
2     Sandwich Place  0.08
3        Coffee Shop  0.06
4    Harbor / Marina  0.03


----COPY-TOP Haussmann - Miromesnil / Imprimerie Paris 8ème----
               venue  freq
0  French Restaurant  0.13
1        

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h1> Results 

In [117]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = paris_grouped['Neighborhood']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Au Top,French Restaurant,Café,Gourmet Shop,Bar,Bakery
1,COPY-TOP Batignolles - Rome / Imprimerie Paris...,Wine Bar,French Restaurant,Hotel,Italian Restaurant,Bakery
2,COPY-TOP Bonne Nouvelle / Imprimerie Paris 10ème,French Restaurant,Hotel,Wine Bar,Japanese Restaurant,Pizza Place
3,COPY-TOP Gare de Lyon - Bastille / Imprimerie ...,Hotel,French Restaurant,Sandwich Place,Coffee Shop,Harbor / Marina
4,COPY-TOP Haussmann - Miromesnil / Imprimerie P...,Hotel,French Restaurant,Wine Shop,Corsican Restaurant,Bar
5,COPY-TOP Le Peletier - Châteaudun / Imprimerie...,French Restaurant,Hotel,Restaurant,Bakery,Japanese Restaurant
6,COPY-TOP Louis Blanc - La Fayette / Imprimerie...,French Restaurant,Café,Bar,Supermarket,Italian Restaurant
7,COPY-TOP Louvre - Etienne Marcel / Imprimerie ...,French Restaurant,Wine Bar,Cocktail Bar,Italian Restaurant,Bakery
8,COPY-TOP Madeleine / Imprimerie Paris 9ème,French Restaurant,Hotel,Boutique,Women's Store,Men's Store
9,COPY-TOP Malesherbes - Saint Augustin / Imprim...,Hotel,French Restaurant,Boutique,Gourmet Shop,Women's Store


In [118]:
neighborhoods_venues_sorted.groupby('1st Most Common Venue').count()

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1st Most Common Venue,,,,,
Bakery,1,1,1,1,1
Bar,3,3,3,3,3
Café,1,1,1,1,1
French Restaurant,22,22,22,22,22
Hotel,11,11,11,11,11
Japanese Restaurant,1,1,1,1,1
Nightclub,1,1,1,1,1
Wine Bar,2,2,2,2,2


<h1> Discussion

Looking at the data, the results show in the neighbourhood Au Top, the 5 common venues were a french restaurant, cafe, gourmet shop, bar and bakery. For Batignolles- Rome the top 5 venues were wine bar, french restaurant, hotel, italian restaurant and bakery. For Bonne Nouvelle it was a french restaurant, hotel, wine bar, japanese restaurant and pizza place. For Gare de Lyon it was a hotel, french restaurant, sandwich place, coffee shop and harbor. Lastly, Haussmann had a hotel, french restaurant, wine shop, corsican restaurant and bar. The common theme across all Paris neighbourhoods were the french restaurants as the top venue

<h1> Conclusion

As an entrepreneur it would be wiser to invest in a French restaurant since it's the most common venue. The second choice would be a hotel.